In [3]:
import numpy as np
from sentence_transformers import SentenceTransformer
from groq import Groq
import os
from dotenv import load_dotenv

load_dotenv()

Groq.api_key = os.getenv("GROQ_API_KEY")

client = Groq()

documents = [
    "Machine learning é um campo da inteligência artificial que permite que computadores aprendam padrões a partir de dados.",
    "O aprendizado de máquina dá aos sistemas a capacidade de melhorar seu desempenho sem serem explicitamente programados.",
    "Em vez de seguir apenas regras fixas, o machine learning descobre relações escondidas nos dados.",
    "Esse campo combina estatística, algoritmos e poder computacional para extrair conhecimento.",
    "O objetivo é criar modelos capazes de generalizar além dos exemplos vistos no treinamento.",
    "Aplicações de machine learning vão desde recomendações de filmes até diagnósticos médicos.",
    "Os algoritmos de aprendizado de máquina transformam dados brutos em previsões úteis.",
    "Diferente de um software tradicional, o ML adapta-se conforme novos dados chegam.",
    "O aprendizado pode ser supervisionado, não supervisionado ou por reforço, dependendo do tipo de problema.",
    "Na prática, machine learning é o motor que impulsiona muitos avanços em visão computacional e processamento de linguagem natural.",
    "Mais do que encontrar padrões, o machine learning ajuda a tomar decisões baseadas em evidências.",
]


In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

doc_embeddings = model.encode(documents) # Esta criando embeddings dos documentos
doc_embeddings

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 39a9d302-2be8-4436-bea9-7934dc257e3a)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


array([[-0.02240201, -0.0445644 , -0.04028781, ...,  0.06722946,
         0.04581729, -0.05725319],
       [ 0.06585719,  0.02475231,  0.00108003, ...,  0.02471357,
         0.0563783 , -0.04562862],
       [-0.07377548, -0.00275126,  0.0095362 , ...,  0.05772354,
         0.05070766, -0.05599884],
       ...,
       [-0.03089636,  0.01383113, -0.01432827, ...,  0.09930875,
         0.04129497,  0.00778692],
       [ 0.01003953, -0.03522453, -0.00955584, ...,  0.07280055,
         0.03746831, -0.0395818 ],
       [-0.05683612, -0.0338407 , -0.03523339, ...,  0.0456608 ,
         0.01687551, -0.03051422]], shape=(11, 384), dtype=float32)

In [14]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [15]:
def retrieve(query, top_k=3):
    query_embedding = model.encode([query])[0]
    
    similarities = []
    for i, doc_emb in enumerate(doc_embeddings):
        sim = cosine_similarity(query_embedding, doc_emb)
        similarities.append((i, sim))
    
    similarities.sort(key=lambda x: x[1], reverse=True) # Ordena pela similaridade e o reverse garante que os mais similares fiquem no topo
    
    return [(documents[i], sim) for i, sim in similarities[:top_k]]

In [16]:
def generate_answer(query, retrieve_docs):
    context = "\n".join([doc for doc, _ in retrieve_docs])
    prompt = f"Contexto:\n{context}\n\nPergunta: {query}\nResposta:"
    
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {
                "role": "system", 
                "content": "Você é um especialista em machine learning. Use apenas o contexto fornecido para responder as perguntas."
            },
            {
                "role": "user", "content": prompt
            },
        ],
        temperature=0,
    )
    
    return response.choices[0].message.content

In [18]:
def rag(query, top_k=3):
    retrieved = retrieve(query, top_k)
    answer = generate_answer(query, retrieved)
    return answer, retrieved

In [19]:
answer, docs = rag("O que é machine learning?")
print("Resposta gerada:\n", answer)

Resposta gerada:
 O machine learning é um campo da inteligência artificial que permite que computadores aprendam padrões a partir de dados, em vez de seguir apenas regras fixas.


In [22]:
for doc, similarity in docs:
    print(f" - {similarity:.3f}: {doc}")

 - 0.730: Machine learning é um campo da inteligência artificial que permite que computadores aprendam padrões a partir de dados.
 - 0.674: Na prática, machine learning é o motor que impulsiona muitos avanços em visão computacional e processamento de linguagem natural.
 - 0.658: Em vez de seguir apenas regras fixas, o machine learning descobre relações escondidas nos dados.


In [ ]:
# v1 = np.array([1,2,3])
# v2 = np.array([4,5,6])

# # calcular produto escalar
# dot_product = np.dot(v1, v2)
# dot_product
# # 1*4 + 2*5 + 3*6 = 32

# # norma euclidiana
# norm_euclidian = np.linalg.norm(v1) * np.linalg.norm(v2)
# norm_euclidian

# # Similaridade cosseno
# dot_product / norm_euclidian


np.float64(0.9746318461970762)